In [2]:
import argparse

import azureml.core
from azureml.core import Experiment,Workspace, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from collections import defaultdict
import cv2
import glob
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy import genfromtxt
import os
import pandas as pd
import pickle
from PIL import Image, ImageFilter
import pydot
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Activation, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import SGD
from skimage import io as io
from skimage import exposure
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh, corner_harris, corner_subpix, corner_peaks, daisy, hog

from sklearn.externals import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
import shutil
import tensorflow as tf
from time import time
import traceback
import timeit
import urllib.request

Using TensorFlow backend.


In [3]:
print(azureml.core.VERSION)

subscription_id = os.getenv("SUBSCRIPTION_ID", default="9bce0414-e6b9-4c79-b146-74018a4b09ac")
resource_group = os.getenv("RESOURCE_GROUP", default="Thesis")
workspace_name = os.getenv("WORKSPACE_NAME", default="Multi-label_classification")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except Exception as e:
    print(e)
    print("Workspace not accessible. Change your parameters or create a new workspace below")
    
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

gpu_cluster_name = "gpucluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                           min_nodes=0,
                                                           max_nodes=4)
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)


1.0.17
Wrote the config file config.json to: C:\Users\olivier.claessen\Desktop\A.I. thesis\aml_config\config.json
Workspace configuration succeeded. Skip the workspace creation steps below
Found existing cpucluster
Found existing gpu cluster


In [4]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

Wrote the config file config.json to: C:\Users\olivier.claessen\Desktop\A.I. thesis\aml_config\config.json


In [5]:
experiment_name = 'multi-class_classification'
exp = Experiment(workspace=ws, name=experiment_name)

In [6]:
# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6") #"STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. gpucluster


In [7]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)
#ds.upload(src_dir='./outputs', target_path='outputs', overwrite=False, show_progress=True)

AzureBlob multilabelclas6598396697 azureml-blobstore-cd752475-573b-4cba-aa3c-7c61a96a4f83


In [8]:
script_folder = './coco-multi-label'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
with open("data/annotations/instances_train2014.json") as read_file:
    train = json.load(read_file)

with open("data/annotations/instances_val2014.json") as read_file:
    val = json.load(read_file)
'''
with open("data/annotations/instances_train2014.json") as read_file:
    instances = json.load(read_file)
with open("data/annotations/person_keypoints_train2014.json") as read_file:
    keypoints = json.load(read_file)
'''

In [9]:
categories = train["categories"]
#print(categories)
train_images = train["images"]
val_images = val["images"]
print("Number of images in 2014 train: "+str(len(train_images)))
print("Number of images in 2014 val: "+str(len(val_images)) +"\n")
train_annotations = train["annotations"]
val_annotations = val["annotations"]
#print(train_annotations[1:20])
val_annotations = val["annotations"]
print("Number of annotations in 2014 train: "+str(len(train_annotations)))
print("Number of annotations in 2014 val: "+str(len(val_annotations)) + "\n")
print("Number of annotations/image in 2014 train: "+str(len(train_annotations)/len(train_images)))
print("Number of annotations/image in 2014 val: " + str(len(val_annotations)/len(val_images)) + "\n")
train_categories = np.zeros(100, dtype=object)
#val_categories = np.zeros(100, dtype=object)
#itemsets = [[] for i in range(581922)]
itemsets = defaultdict(list)
for annotation in train_annotations:
    train_categories[annotation['category_id']] += 1
    itemsets[annotation['image_id']].append(annotation['category_id'])
unique_itemsets = []
for entry in itemsets:
    if(entry not in unique_itemsets):
        unique_itemsets.append(entry)
print(len(unique_itemsets))
'''
for annotation in val_annotations:
    val_categories[annotation['category_id']] += 1
train_categories = train_categories[train_categories != 0]
val_categories = val_categories[val_categories != 0]
'''
print("Label density in 2014 train: "+str(len(train_annotations)/len(train_categories)/len(train_images)))
#print("Label density in 2014 val: "+str(len(val_annotations)/len(val_categories)/len(val_images)))
print(train.keys())
print("\n")
#print(images[1:10])
#print("\n")
#print(annotations[1:10])
#print("\n")
#print(categories)
#for i in range(0, len(train_categories)):
#    print(str(categories[i]["id"]) + categories[i]["name"] + ": " + str(train_categories[i]))
#    train_categories[i] = (categories[i]["name"], train_categories[i])
#for i in range(0, len(val_categories)):
    #print(categories[i]["name"] + ": " + str(train_categories[i]))
#    val_categories[i] = (categories[i]["name"], val_categories[i])
plt.figure(1)
plt.subplot(211)
plt.plot([i[1] for i in train_categories])
#plt.subplot(212)
#plt.plot([i[1] for i in val_categories])
#plt.show()
train_categories = sorted(train_categories, key=lambda tup: tup[1], reverse=False)
#val_categories = sorted(val_categories, key=lambda tup: tup[1], reverse=False)
print("\n10 least used features \n")
print(train_categories[0:9])
print("\n")
#print(val_categories[0:9])
print("\n10 most used features \n")
print(train_categories[-10:])
print("\n")
#print(val_categories[-10:])

Number of images in 2014 train: 82783
Number of images in 2014 val: 40504

Number of annotations in 2014 train: 604907
Number of annotations in 2014 val: 291875

Number of annotations/image in 2014 train: 7.307140354903785
Number of annotations/image in 2014 val: 7.206078412008691

82081
Label density in 2014 train: 0.07307140354903784
dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])


1person: 0
2bicycle: 185316
3car: 4955
4motorcycle: 30785
5airplane: 6021
6bus: 3833
7train: 4327
8truck: 3159
9boat: 7050
10traffic light: 7590
11fire hydrant: 9159
13stop sign: 1316
14parking meter: 0
15bench: 1372
16bird: 833
17cat: 6751
18dog: 7290
19horse: 3301
20sheep: 3774
21cow: 4666
22elephant: 6654
23bear: 5686
24zebra: 3905
25giraffe: 903
27backpack: 3685
28umbrella: 3596
31handbag: 0
32tie: 6200
33suitcase: 7865
34frisbee: 0
35skis: 0
36snowboard: 8778
37sports ball: 4497
38kite: 4251
39baseball bat: 1862
40baseball glove: 4698
41skateboard: 1960
42surfboard: 4392
43tenn

IndexError: list index out of range

In [ ]:
import sys
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [ ]:
image_id = 49 #9,25,30,34,36,49
def findAnnotations(id, property):
    items = []
    for annotation in train_annotations: 
        if annotation[property] == id:
            items.append(annotation)
    return items
items = findAnnotations(image_id, 'image_id')

In [ ]:
def findValueDictionary(dict,key,value, returnkey):
    for entry in dict:
        if entry[key] == value:
            return entry[returnkey]

image_id_string = str(image_id).zfill(12)
image = np.array(Image.open('data/train2014/COCO_train2014_'+ image_id_string +'.jpg'), dtype=np.uint8)
fig,ax = plt.subplots(1)
ax.imshow(image)
colors = np.random.random((len(categories)+10, 3))
i=0
bbox_list = []
item_list = []
for item in items:
    label = findValueDictionary(categories, 'id', item['category_id'], 'name')
    rect = patches.Rectangle((item['bbox'][0], (item['bbox'][1])), item['bbox'][2], item['bbox'][3],linewidth=1,edgecolor=colors[item['category_id']-1],facecolor='none', label= label)
    ax.add_patch(rect)
    bbox_list.append(rect)
    item_list.append(item['category_id'])
#print(items)
plt.figure(figsize=(3,4))
plt.legend(handles=bbox_list)
plt.show()

In [10]:
import argparse

import azureml.core
from azureml.core import Run
import cv2
import numpy as np
from numpy import genfromtxt
import os
from PIL import Image, ImageFilter
from skimage import io as io
from skimage import exposure
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh, corner_harris, corner_subpix, corner_peaks, daisy, hog

from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from time import time

def loadAnnotations(fileLocation):
    with open(fileLocation) as read_file:
        annotations = json.load(read_file)
    return annotations
    
def getBoundingBox(x,y,w,h,img):
    return img[y:y+h,x:x+w]

def findAnnotations(id, property, annotations):
    items = []
    for annotation in annotations: 
        if annotation[property] == id:
            items.append(annotation)
    return items

def zca_whitening_matrix(X):
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True) # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U,S,V = np.linalg.svd(sigma)
        # U: [M x M] eigenvectors of sigma.
        # S: [M x 1] eigenvalues of sigma.
        # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 1e-5
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0/np.sqrt(S + epsilon)), U.T)) # [M x M]
    return ZCAMatrix

def preProcessImage(img):
    img = np.asarray(img)
    #img = rgb2gray(img)
    #img = cv2.GaussianBlur(img,(5,5),0)
    #img = cv2.medianBlur(img,5)
    #img = cv2.bilateralFilter(img,9,75,75)
    #img = cv2.blur(img,(5,5))
    #kernel = np.ones((5,5),np.float32)/25
    #img = cv2.filter2D(img,-1,kernel)
    return img

def noisy(image):
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.004
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
          for i in image.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
          for i in image.shape]
    out[coords] = 0
    return out

def getBoundingBoxesPictures(annotations, path):
    bounded_images = []
    bounded_annotations = []
    try:
        for annotation in annotations:
            image_id = annotation['image_id']
            image_id_string = str(image_id).zfill(12)
            image = Image.open(path + image_id_string +'.jpg').convert('RGB')
            image = preProcessImage(image) 
            image_resized = resize(getBoundingBox(int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3]),image)
                                   , (48, 48),
                           anti_aliasing=True)
            bounded_images.append(image_resized)
            bounded_annotations.append(annotation['category_id'])     
    except Exception as ex:
            print(ex)   
    bounded_images = np.asarray(bounded_images)
    bounded_annotations = np.asarray(bounded_annotations)
    return bounded_images,bounded_annotations

def calculateHogFeatures(gray_image, o, pixels, cells):
    features = hog(gray_image, orientations=o, 
                              pixels_per_cell=(pixels, pixels),
                              cells_per_block=(cells, cells), 
                              transform_sqrt=True, 
                              visualize=False, block_norm = "L2-Hys")
    return features
def calculateDaisyFeatures(gray_image):
    descs = daisy(gray_image, step=180, radius=15, rings=3, histograms=6,
                         orientations=8, visualize=False)
    descs_num = descs.shape[0] * descs.shape[1]
    return descs.reshape(descs.size).tolist()

def calculateDoG(gray_image):
    blobs_dog = blob_dog(gray_img, max_sigma=30, threshold=.1)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
    return blobs_dog

def calculateSIFT(sift, gray_image):
    kp = sift.detect(gray_image,None)
    return kp

def calculateFeatures(imgs):
    #sift = cv2.xfeatures2d.SIFT_create()
    hog_features = []
    dog_features = []
    daisy_features = []
    sift_features = []
    i = 0;
    for img in imgs:
        hog_features.append(calculateHogFeatures(img,8,16,1))
        #dog_features.append(calculateDoG(img))
        #sift_features.append(calculateSIFT(sift, img))
        daisy_features.append(calculateDaisyFeatures(img))
    return hog_features, daisy_features

def svmFit(x_train, y_train):
    print("Fitting the classifier to train")
    t0 = time()
    param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'),
                       param_grid, cv=5)
    clf = clf.fit(x_train, y_train)
    print("Completed in %0.3fs" % (time() - t0))
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)
    return clf

def saveModel(model, filename):
    print("Saving file...")
    joblib.dump(model, open(filename, 'wb'))
    print("File saved")
    
def loadModel(filename):
    print("loading file...")
    joblib.load(filename)
    print("Model loaded")
    
def svmPredict(x_test, y_test, model):
    print("Predicting the test set")
    t0 = time()
    y_pred = model.predict(x_test)
    print("Completed in %0.3fs" % (time() - t0))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    
def randomForestFit(x_train, y_train, estimators):
    print("Fitting the classifier to train")
    t0 = time()
    rf = RandomForestClassifier(n_estimators=estimators)
    rf.fit(x_train, y_train);
    print("Completed in %0.3fs" % (time() - t0))
    return rf

def rfPredict(x_test, y_test, model):
    print("Predicting the test set")
    t0 = time()
    y_pred = model.predict(x_test)
    print("Completed in %0.3fs" % (time() - t0))
    errors = abs(y_pred - y_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))

  
def constructCNN(conv_layer, layer_size, kernel_size, dense_size, dense_layer, dropout, num_classes, x_train):
    model = Sequential()

    model.add(Conv2D(layer_size, (kernel_size, kernel_size), input_shape=x_train.shape[1:]))

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (kernel_size, kernel_size)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    if(dropout):
        model.add(Dropout(0.25))
    model.add(Flatten())

    for j in range(dense_layer):
        model.add(Dense(dense_size))
        model.add(Activation('relu'))
    if(dropout):
        model.add(Dropout(0.25))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model
    
# Create a function called "chunks" with two arguments, l and n:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def splitPreprocessing(splits, data_folder, path):
    labels = []
    features = np.asarray([[],[]])
    for split in splits:
        bboxes = getBoundingBoxesPictures(split, os.path.join(data_folder, path))
        splitfeatures = calculateFeatures(bboxes[0])
        features = np.append(features, splitfeatures,1)
        labels.extend(bboxes[1])
    return labels, features.tolist()

data_folder = 'data'

path = "train2014/COCO_train2014_"
valpath = "val2014/val2014/COCO_val2014_"
category = "category_id"

In [87]:
print(x_test.shape)

(17457, 48, 48, 3)


In [23]:
'''
train_annotations = loadAnnotations(os.path.join(data_folder, 'annotations/instances_train2014.json'))
train_annotations = train_annotations["annotations"]

val_annotations = loadAnnotations(os.path.join(data_folder, 'annotations/instances_val2014.json'))
val_annotations = val_annotations["annotations"]
'''
'''
#testsplits = list(chunks(dummy_annotations, 50))
#labels, features = splitPreprocessing(testsplits, data_folder, path)

'''

categories_subset = [2,4]
bbox = getBoundingBoxesPictures(findAnnotations(categories_subset[0], category, train_annotations),  os.path.join(data_folder, path))
imgs = bbox[0]
categories_subset.pop(0)
labels = bbox[1]
for category_id in categories_subset:
    bbox = getBoundingBoxesPictures(findAnnotations(category_id, category, train_annotations),  os.path.join(data_folder, path))
    imgs = np.concatenate((imgs,bbox[0]))
    labels = np.concatenate((labels,bbox[1]))

features = calculateFeatures(imgs)
x_trainfeat, x_testfeat, y_trainfeat, y_testfeat = train_test_split(
    features, y_train, stratify=y_train,  test_size=0.25, random_state=42)
svm = svmFit(x_trainfeat, y_trainfeat)

svmResults = svmPredict(x_testfeat, y_testfeat, svm)

'''
x_trainfeat, x_testfeat, y_trainfeat, y_testfeat = train_test_split(
    features[1], y_train, stratify=y_train, test_size=0.25, random_state=42)
svmDaisy = svmFit(x_trainfeat, y_trainfeat)
svmDaisyResults = svmPredict(x_testfeat, y_testfeat, svmDaisy)
#saveModel(svm, 'SVM_DAISY.sav')
os.makedirs('outputs', exist_ok=True)
'''

(72,)
(152,)


ValueError: operands could not be broadcast together with shapes (72,) (152,) 

In [67]:
'''
categories_subset = [2,4]#[2,4,5,6,7]
bbox = getBoundingBoxesPictures(findAnnotations(categories_subset[0], category, train_annotations),  os.path.join(data_folder, path))
imgs = bbox[0]
categories_subset.pop(0)
labels = bbox[1]
for category_id in categories_subset:
    bbox = getBoundingBoxesPictures(findAnnotations(category_id, category, train_annotations),  os.path.join(data_folder, path))
    imgs = np.concatenate((imgs,bbox[0]))
    labels = np.concatenate((labels,bbox[1]))
'''
features = calculateFeatures(imgs)

x_trainfeat, x_testfeat, y_trainfeat, y_testfeat = train_test_split(
    features[0], y_train, test_size=0.25, random_state=42)
rfHOG = randomForestFit(x_trainfeat, y_trainfeat, 1000)
rfHOG_results = rfPredict(x_testfeat, y_testfeat, rfHOG)
x_trainfeat, x_testfeat, y_trainfeat, y_testfeat = train_test_split(
    features[1], y_train, test_size=0.25, random_state=42)
rfDaisy = randomForestFit(x_trainfeat, y_trainfeat, 1000)
rfDaisy_results = rfPredict(x_testfeat, y_testfeat, rfDaisy)

Fitting the classifier to train
Completed in 289.998s
Predicting the test set
Completed in 4.329s
Accuracy: 0.5256624963057827
Fitting the classifier to train
Completed in 451.803s
Predicting the test set
Completed in 4.565s
Accuracy: 0.5697960792040193


In [16]:
# width 578.2023976316078
# heigth 483.5494085111049

path = "data/train2014/COCO_train2014_"
def getPictures(itemsets, path, instance):
    images = []
    image_ids = []
    image_tags = []
    avg_width, avg_height = (0,0)
    try:
        for itemset in itemsets:
            print(itemset)
            print(itemsets[itemset])
            if(instance in itemsets[itemset]):
                image_id_string = str(itemset).zfill(12)
                image_ids.append(str(itemset))
                image = Image.open(path + image_id_string +'.jpg')
                processed_img = preProcessImage(image)
                image.close()
                resized_image = resize(processed_img, (350, 292),anti_aliasing=True)
                images.append(resized_image)
                image_tags.append(itemsets[itemset])
    except Exception as ex:
            print(ex)   
    return image_ids,images,image_tags

pictureData = getPictures(itemsets, path, 18)
#print(pictureData)

NameError: name 'itemsets' is not defined

In [11]:
def example_errors():
    generator_test.reset()
    
    # Predict the classes for all images in the test-set.
    y_pred = new_model.predict_generator(generator_test,
                                         steps=steps_test)

    # Convert the predicted classes from arrays to integers.
    cls_pred = np.argmax(y_pred,axis=1)

    # Plot examples of mis-classified images.
    plot_example_errors(cls_pred)
    
    # Print the confusion matrix.
    print_confusion_matrix(cls_pred)
categories_subset = [2,6,19]
bbox = getBoundingBoxesPictures(findAnnotations(categories_subset[0], category, train_annotations),  os.path.join(data_folder, path))
bbox_val = getBoundingBoxesPictures(findAnnotations(categories_subset[0], category, val_annotations),  os.path.join(data_folder, valpath))
imgs = bbox[0]
imgs_val = bbox_val[0]
y_train = bbox[1]
y_test = bbox_val[1]
categories_subset.pop(0)
for category_id in categories_subset:
    bbox = getBoundingBoxesPictures(findAnnotations(category_id, category, train_annotations),  os.path.join(data_folder, path))
    imgs = np.concatenate((imgs,bbox[0]))
    y_train = np.concatenate((y_train,bbox[1]))
    bbox_val = getBoundingBoxesPictures(findAnnotations(category_id, category, val_annotations),  os.path.join(data_folder, valpath))
    imgs_val = np.concatenate((imgs_val,bbox_val[0]))
    y_test = np.concatenate((y_test,bbox_val[1]))

#images = np.asarray(pictureData[1])

'''
#x_train, x_test, y_train, y_test = train_test_split(
#    imgs, y_train, test_size=0.25, random_state=42)
'''
x_train = imgs/255
x_test = imgs_val/255

for (i,value) in enumerate(y_train):
    if value == 2:
        y_train[i] = 0
        continue
    if value == 6:
        y_train[i] = 1
        continue
    if value == 19:
        y_train[i] = 2
        continue
    if value == 20:
        y_train[i] = 3
        continue
    if value == 33:
        y_train[i] = 4
        continue
    if value == 46:
        y_train[i] = 5
        continue
    if value == 49:
        y_train[i] = 6
        continue
    if value == 55:
        y_train[i] = 7
        continue
for (i,value) in enumerate(y_test):
    if value == 2:
        y_test[i] = 0
        continue
    if value == 6:
        y_test[i] = 1
        continue
    if value == 19:
        y_test[i] = 2
        continue
    if value == 20:
        y_test[i] = 3
        continue
    if value == 33:
        y_test[i] = 4
        continue
    if value == 46:
        y_test[i] = 5
        continue
    if value == 49:
        y_test[i] = 6
        continue
    if value == 55:
        y_test[i] = 7
        continue

datagen = ImageDataGenerator(
    zca_whitening=True, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
datagen.fit(x_train)
datagen_test = ImageDataGenerator(zca_whitening=True)
datagen_test.fit(x_test)         

ylabels_test = to_categorical(y_test)
ylabels_train = to_categorical(y_train)
#joblib.dump(value=ylabels_train, filename='outputs/ylabels_train2,6,19,20,33,46,49,55c.npy')
#joblib.dump(value=ylabels_test, filename='outputs/ylabels_test2,6,19,20,33,46,49,55c.npy')

c:\users\olivier.claessen\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
c:\users\olivier.claessen\appdata\local\programs\python\python36\lib\site-packages\keras_preprocessing\image\image_data_generator.py:334: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [33]:
x_train = joblib.load('outputs/x_train2,6,19,20,33,46c.npy')
x_test = joblib.load('outputs/x_test2,6,19,20,33,46c.npy')
ylabels_train = joblib.load('outputs/y_testlabels2,6,19,20,33,46.npy')
ylabels_test = joblib.load('outputs/y_trainlabels2,6,19,20,33,46.npy')

In [34]:
datagen = ImageDataGenerator(
    featurewise_center=True, 
    zca_whitening=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
datagen.fit(x_train)
datagen_test = ImageDataGenerator(featurewise_center=True, 
    zca_whitening=True,)
datagen_test.fit(x_test)    

c:\users\olivier.claessen\appdata\local\programs\python\python36\lib\site-packages\keras_preprocessing\image\image_data_generator.py:339: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [35]:
sess = tf.Session()
early_stopping_monitor = EarlyStopping(patience = 3)



dense_layers = [1]
layer_sizes = [64]
dense_sizes = [64]
conv_layers = [1]
kernel_sizes = [3]
for dense_layer in dense_layers:
    for dense_size in dense_sizes:
        for kernel_size in kernel_sizes:
            for layer_size in layer_sizes:
                for conv_layer in conv_layers:
                    model = constructCNN(conv_layer, layer_size, kernel_size, dense_size, dense_layer, True, 3, x_train)
                    NAME = "{}-conv-{}-nodes-{}-dense-{}-dense_size-{}-kernel-{}".format(conv_layer, layer_size, dense_layer, dense_size, kernel_size, int(time()))
                    print(NAME)
                    tensorboard = TensorBoard(log_dir="logs2,6,19,20,33,46,49,55c/{}".format(NAME))
                    #model.fit(x_train, ylabels_train.astype(np.float32), batch_size=70, epochs=100, validation_data = (x_test, ylabels_test), callbacks = [tensorboard],shuffle=True)
                    model.fit_generator(datagen.flow(x_train, ylabels_train, batch_size=32,shuffle=True), 
                                        epochs=15, callbacks = [early_stopping_monitor], validation_data = (x_test, ylabels_test))

'''
y_pred = model.predict_classes(x_test)
true_preds = [(x,y) for (x,y,p) in zip(X_test, y_test, y_pred) if y == p]
false_preds = [(x,y,p) for (x,y,p) in zip(X_test, y_test, y_pred) if y != p]
print("Number of true predictions: ", len(true_preds))
print("Number of false predictions:", len(false_preds))
'''
'''
model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=x_train.shape[1:], activation = 'relu'))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(8))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
        
print("2x3x3x128,2x3x3x64,0.25,D64,0.25 model.fit")
model.fit(x_train, ylabels_train.astype(np.float32), batch_size=70, epochs=100, validation_data = (x_test, ylabels_test), callbacks = [])
'''

1-conv-64-nodes-1-dense-64-dense_size-3-kernel-1554217025
Epoch 1/15
436/436 [==============================] - 308s 707ms/step - loss: 0.8386 - acc: 0.6225 - val_loss: 1.1511 - val_acc: 0.3695
Epoch 2/15
159/436 [=========>....................] - ETA: 2:56 - loss: 0.6977 - acc: 0.6994

KeyboardInterrupt: 

In [ ]:
%%writefile $script_folder/train.py

import argparse

import azureml.core
from azureml.core import Run

import numpy as np
from numpy import genfromtxt
import os
from PIL import Image, ImageFilter
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh, corner_harris, corner_subpix, corner_peaks, daisy, hog

from sklearn.externals import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
import time

def loadAnnotations(fileLocation):
    with open(fileLocation) as read_file:
        annotations = json.load(read_file)
    return annotations
    
def getBoundingBox(x,y,w,h,img):
    return img[y:y+h,x:x+w]

def findAnnotations(id, property, annotations):
    items = []
    for annotation in annotations: 
        if annotation[property] == id:
            items.append(annotation)
    return items

def preProcessImage(img):
    img = np.asarray(img)
    img = rgb2gray(img)
    #img = cv2.GaussianBlur(img,(5,5),0)
    #img = cv2.medianBlur(img,5)
    #img = cv2.bilateralFilter(img,9,75,75)
    #img = cv2.blur(img,(5,5))
    #kernel = np.ones((5,5),np.float32)/25
    #img = cv2.filter2D(img,-1,kernel)
    return img

def getBoundingBoxesPictures(annotations, path):
    bounded_images = []
    bounded_annotations = []
    try:
        for annotation in annotations:
            image_id = annotation['image_id']
            image_id_string = str(image_id).zfill(12)
            image = Image.open(path + image_id_string +'.jpg')
            image = preProcessImage(image) 
            image_resized = resize(getBoundingBox(int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3]),image)
                                   , (64, 64),
                           anti_aliasing=True)
            bounded_images.append(image_resized)
            bounded_annotations.append(annotation['category_id'])     
    except Exception as ex:
            print(ex)   
    bounded_images = np.asarray(bounded_images)
    bounded_annotations = np.asarray(bounded_annotations)
    return bounded_images,bounded_annotations

def calculateHogFeatures(gray_image, o, pixels, cells):
    features = hog(gray_image, orientations=o, 
                              pixels_per_cell=(pixels, pixels),
                              cells_per_block=(cells, cells), 
                              transform_sqrt=True, 
                              visualize=False, block_norm = "L2-Hys")
    return features
def calculateDaisyFeatures(gray_image):
    descs = daisy(gray_image, step=180, radius=15, rings=3, histograms=6,
                         orientations=8, visualize=False)
    descs_num = descs.shape[0] * descs.shape[1]
    return descs.reshape(descs.size).tolist()

def calculateDoG(gray_image):
    blobs_dog = blob_dog(gray_img, max_sigma=30, threshold=.1)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
    return blobs_dog

def calculateSIFT(sift, gray_image):
    kp = sift.detect(gray_image,None)
    return kp

def calculateFeatures(imgs):
    #sift = cv2.xfeatures2d.SIFT_create()
    hog_features = []
    dog_features = []
    daisy_features = []
    sift_features = []
    i = 0;
    for img in imgs:
        hog_features.append(calculateHogFeatures(img,8,16,1))
        #dog_features.append(calculateDoG(img))
        #sift_features.append(calculateSIFT(sift, img))
        daisy_features.append(calculateDaisyFeatures(img))
    return hog_features, daisy_features

def svmFit(x_train, y_train):
    print("Fitting the classifier to train")
    t0 = time.time()
    param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'),
                       param_grid, cv=5)
    clf = clf.fit(x_train, y_train)
    print("Completed in %0.3fs" % (time.time() - t0))
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)
    return clf

def saveModel(model, filename):
    print("Saving file...")
    joblib.dump(model, open(filename, 'wb'))
    print("File saved")
    
def loadModel(filename):
    print("loading file...")
    model = joblib.load(filename)
    print("Model loaded")
    return model
    
def svmPredict(x_test, y_test, model):
    print("Predicting the test set")
    t0 = time.time()
    y_pred = model.predict(x_test)
    print("Completed in %0.3fs" % (time.time() - t0))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))
    
def randomForestFit(x_train, y_train, estimators, random_state):
    print("Fitting the classifier to train")
    t0 = time.time()
    rf = RandomForestRegressor(n_estimators = estimators, random_state = random_state)
    rf.fit(x_train, y_train);
    print("Completed in %0.3fs" % (time.time() - t0))
    return rf

def rfPredict(x_test, y_test, model):
    print("Predicting the test set")
    t0 = time.time()
    y_pred = model.predict(x_test)
    print("Completed in %0.3fs" % (time.time() - t0))
    errors = abs(y_pred - y_test)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    
    
# Create a function called "chunks" with two arguments, l and n:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def splitPreprocessing(splits, data_folder, path):
    labels = []
    features = np.asarray([[],[]])
    for split in splits:
        bboxes = getBoundingBoxesPictures(split, os.path.join(data_folder, path))
        splitfeatures = calculateFeatures(bboxes[0])
        features = np.append(features, splitfeatures,1)
        labels.extend(bboxes[1])
    return labels, features.tolist()

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()
data_folder = os.path.join(args.data_folder, 'data')
print('Data folder:', data_folder)
run = Run.get_context()

path = "train2014/COCO_train2014_"
category = "category_id"
train_annotations = loadAnnotations(os.path.join(data_folder, 'annotations/instances_train2014.json'))
train_annotations = train_annotations["annotations"]
'''
dummy_annotations = np.concatenate((findAnnotations(87, category, train_annotations),findAnnotations(89, category, train_annotations)))
bbox_train = getBoundingBoxesPictures(dummy_annotations, os.path.join(data_folder, path))

'''
categories_subset = [2,6,19,20,33,46,49,55]
bbox = getBoundingBoxesPictures(findAnnotations(categories_subset[0], category, train_annotations),  os.path.join(data_folder, path))
imgs = bbox[0]
categories_subset.pop(0)
labels = bbox[1]
for category_id in categories_subset:
    bbox = getBoundingBoxesPictures(findAnnotations(category_id, category, train_annotations),  os.path.join(data_folder, path))
    imgs = np.concatenate((imgs,bbox[0]))
    labels = np.concatenate((labels,bbox[1]))
features = calculateFeatures(imgs)

#splits = list(chunks(train_annotations, 20000))
#labels, features = splitPreprocessing(splits, data_folder, path)
print("[2,6,19,20,33,46,49,55]")
x_train, x_test, y_train, y_test = train_test_split(
    features[0], labels, test_size=0.25, random_state=42)
print("HOG")
svm = svmFit(x_train, y_train)
#saveModel(svm, 'SVM_DAISY.sav')
run.log('prediction', svmPredict(x_test, y_test, svm))
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=svm, filename='outputs/SVM.pkl')
joblib.dump(value=features, filename='outputs/features.npy')

x_train, x_test, y_train, y_test = train_test_split(
    features[1], labels, test_size=0.25, random_state=42)
print("DAISY")
svm = svmFit(x_train, y_train)
#saveModel(svm, 'SVM_DAISY.sav')
run.log('prediction', svmPredict(x_test, y_test, svm))

In [36]:
%%writefile $script_folder/neural_network.py

import argparse

import azureml.core
from azureml.core import Run

import numpy as np
from numpy import genfromtxt
import os
from PIL import Image, ImageFilter
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Activation, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import SGD
from skimage.transform import resize

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib
import tensorflow as tf
import time

def loadAnnotations(fileLocation):
    with open(fileLocation) as read_file:
        annotations = json.load(read_file)
    return annotations
    
def getBoundingBox(x,y,w,h,img):
    return img[y:y+h,x:x+w]

def findAnnotations(id, property, annotations):
    items = []
    for annotation in annotations: 
        if annotation[property] == id:
            items.append(annotation)
    return items

def preProcessImage(img):
    img = np.asarray(img)
    return img

def getBoundingBoxesPictures(annotations, path):
    bounded_images = []
    bounded_annotations = []
    try:
        for annotation in annotations:
            image_id = annotation['image_id']
            image_id_string = str(image_id).zfill(12)
            image = Image.open(path + image_id_string +'.jpg').convert('RGB')
            image = preProcessImage(image) 
            image_resized = resize(getBoundingBox(int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3]),image)
                                   , (48, 48),
                           anti_aliasing=True)
            bounded_images.append(image_resized)
            bounded_annotations.append(annotation['category_id'])     
    except Exception as ex:
            print(ex)   
    bounded_images = np.asarray(bounded_images)
    bounded_annotations = np.asarray(bounded_annotations)
    return bounded_images,bounded_annotations

 
def constructCNN(conv_layer, layer_size, kernel_size, dense_size, dense_layer, dropout, num_classes, x_train):
    model = Sequential()

    model.add(Conv2D(layer_size, (kernel_size, kernel_size), input_shape=x_train.shape[1:]))

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (kernel_size, kernel_size)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    if(dropout):
        model.add(Dropout(0.25))
    model.add(Flatten())

    for j in range(dense_layer):
        model.add(Dense(dense_size))
        model.add(Activation('relu'))
    if(dropout):
        model.add(Dropout(0.25))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()
data_folder = os.path.join(args.data_folder, 'outputs')
print('Data folder:', data_folder)
run = Run.get_context()

path = "train2014/COCO_train2014_"
category = "category_id"
x_train = joblib.load(os.path.join(data_folder, 'trainset2,6,19,20,33,46,49,55c.npy'))
x_test = joblib.load(os.path.join(data_folder, 'testset2,6,19,20,33,46,49,55c.npy'))
ylabels_train = joblib.load(os.path.join(data_folder, 'ylabels_train2,6,19,20,33,46,49,55.npy'))
ylabels_test = joblib.load(os.path.join(data_folder, 'ylabels_test2,6,19,20,33,46,49,55.npy'))


datagen = ImageDataGenerator(
    featurewise_center=True, 
    featurewise_std_normalization=True,
    zca_whitening=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
datagen.fit(x_train)
datagen_test = ImageDataGenerator(featurewise_center=True, 
    featurewise_std_normalization=True,
    zca_whitening=True,)
datagen_test.fit(x_test)       

sess = tf.Session()
early_stopping_monitor = EarlyStopping(patience = 3)



dense_layers = [3]
layer_sizes = [64]
dense_sizes = [64]
conv_layers = [3]
kernel_sizes = [3]
for dense_layer in dense_layers:
    for dense_size in dense_sizes:
        for kernel_size in kernel_sizes:
            for layer_size in layer_sizes:
                for conv_layer in conv_layers:
                    model = constructCNN(conv_layer, layer_size, kernel_size, dense_size, dense_layer, True, 8, x_train)
                    NAME = "{}-conv-{}-nodes-{}-dense-{}-dense_size-{}-kernel".format(conv_layer, layer_size, dense_layer, dense_size, kernel_size)
                    print(NAME)

                    #model.fit(x_train, ylabels_train.astype(np.float32), batch_size=70, epochs=500, validation_data = (x_test, ylabels_test), callbacks = [early_stopping_monitor])
                    model.fit_generator(datagen.flow(x_train, ylabels_train, batch_size=32,shuffle=True), 
                                        epochs=100, callbacks = [], validation_data = datagen_test.flow(x_test, ylabels_test, shuffle=True))

Overwriting ./coco-multi-label/neural_network.py


In [37]:
import shutil
shutil.copy('utils.py', script_folder)

'./coco-multi-label\\utils.py'

In [ ]:
from azureml.train.estimator import Estimator


script_params = {
    '--data-folder': ds.as_mount(),
}

'''
est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn', 'scikit-image', 'pillow', 'opencv', 'numpy'])

run = exp.submit(config=est)
run
'''

keras_est = TensorFlow(source_directory=script_folder,
                       script_params=script_params,
                       compute_target=compute_target,
                       entry_script='neural_network.py',
                       pip_packages=['keras', 'scikit-learn', 'scikit-image', 'pillow'],
                      use_gpu=True)

run = exp.submit(config=keras_est)
run

from azureml.widgets import RunDetails
RunDetails(run).show()

run.wait_for_completion(show_output=True) # specify True for a verbose log

print(run.get_metrics())

print(run.get_file_names())

# register model 
#model = run.register_model(model_name='svm_hog', model_path='outputs/svm_hog.pkl')
#print(model.name, model.id, model.version, sep = '\t')

In [19]:
for r in exp.get_runs():  
    print(r.id, r.get_status())
    if r.get_status() not in ['Complete', 'Failed']:
        r.cancel()

# if you know the run id, you can "rehydrate" the run
from azureml.core import get_run
#r = get_run(experiment=exp, run_id="multi-class_classification_1552234551_7c1e71ae", rehydrate=True)
# check the returned run type and status
print(type(r), r.get_status())

# you can cancel a run if it hasn't completed or failed
#if r.get_status() not in ['Complete', 'Failed']:
    #r.cancel()

multi-class_classification_1553865295_69cd11d7 Running
multi-class_classification_1553865088_1aa09112 Running
multi-class_classification_1553864091_b5d7a8f0 Completed
multi-class_classification_1553862335_1cc89eea Canceled
multi-class_classification_1553861876_d7b23c93 Canceled
multi-class_classification_1553861597_9c5e9bb1 Completed
multi-class_classification_1553861290_d368a010 Canceled
multi-class_classification_1553860670_3301962e Completed
multi-class_classification_1553860550_0d3885a6 Completed
multi-class_classification_1553860374_5df4ed84 Canceled
multi-class_classification_1553860236_685bc22d Completed
multi-class_classification_1553859814_0f6fbc92 Completed
multi-class_classification_1553859524_df3f497b Completed
multi-class_classification_1553858058_125fcfa9 Canceled
multi-class_classification_1553856813_a97f9ba4 Completed
multi-class_classification_1553782189_1323af86 Failed
multi-class_classification_1553780005_dac75afb Completed
multi-class_classification_1553776065_265e7

In [ ]:
gray_img = rgb2gray(image)
color = ('b','g','r')
plt.figure()
for i,col in enumerate(color):
    histr = cv2.calcHist([image],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()


In [ ]:
#df.head()
#print(x_train[0:9].values)
#print(y_train[0:9])

In [ ]:
#print(features[0][0])
#print(features[1])
'''
fig, ax = plt.subplots(1, figsize=(8, 4))

hog_image_rescaled = exposure.rescale_intensity(features[0][0][1], in_range=(0, 10))

ax.axis('off')
ax.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax.set_title('Histogram of Oriented Gradients')
plt.show()
'''

In [ ]:
'''
blobs_log = blob_log(gray_img, max_sigma=30, num_sigma=10, threshold=.1)

blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(gray_img, max_sigma=30, threshold=.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(gray_img, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image, interpolation='nearest')
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()
'''

In [ ]:
'''
fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)

fig, ax = plt.subplots(1, figsize=(8, 4))

hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax.axis('off')
ax.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax.set_title('Histogram of Oriented Gradients')
plt.show()
'''

In [ ]:
'''
print(gray_img)
coords = corner_peaks(corner_harris(gray_img), min_distance=5)

fig, (ax) = plt.subplots(1, figsize=(8, 4))

ax.axis('off')
ax.imshow(image)
ax.plot(coords[:, 1], coords[:, 0], 'or', ms=4)
'''

In [ ]:
'''
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)

siftimg=cv2.drawKeypoints(gray,kp,img)
plt.figure()
plt.axis('off')
plt.imshow(siftimg)
plt.show()
'''